In [1]:
from google.colab import files
files.upload()  # Upload kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohanapriya9080","key":"04d2427efb549c737249b6fbae6288cb"}'}

In [2]:
import os
import zipfile

# Create .kaggle directory and move kaggle.json there
os.makedirs("/root/.kaggle", exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d spscientist/students-performance-in-exams

# Unzip the dataset
with zipfile.ZipFile("students-performance-in-exams.zip", "r") as zip_ref:
    zip_ref.extractall("student_data")


Dataset URL: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams
License(s): unknown
  0% 0.00/8.70k [00:00<?, ?B/s]
100% 8.70k/8.70k [00:00<00:00, 34.8MB/s]


In [3]:
import pandas as pd

# Load CSV
df = pd.read_csv("student_data/StudentsPerformance.csv")
df.head()


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# Add a 'pass' column: pass if average score >= 50
df["average_score"] = df[['math score', 'reading score', 'writing score']].mean(axis=1)
df["pass"] = df["average_score"].apply(lambda x: 1 if x >= 50 else 0)

# Encode categorical variables
df_encoded = pd.get_dummies(df.drop(["average_score"], axis=1), drop_first=True)

# Separate features and target
X = df_encoded.drop("pass", axis=1)
y = df_encoded["pass"]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)


In [6]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      1.00      1.00       173

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [9]:
# Example: Male, standard lunch, test prep completed, math=45, read=50, write=55
# Create a dictionary with all the columns from X_train and set values for the example case
new_data = {col: [0] for col in X_train.columns}

new_data['math score'] = [45]
new_data['reading score'] = [50]
new_data['writing score'] = [55]
new_data['gender_male'] = [1] # Male
new_data['race/ethnicity_group C'] = [1] # Assuming group C for this example
new_data['parental level of education_some college'] = [1] # Some college
new_data['lunch_standard'] = [1] # Standard lunch
new_data['test preparation course_completed'] = [1] # Test prep completed


new_df = pd.DataFrame(new_data)

# Ensure the order of columns matches X_train
new_df = new_df[X_train.columns]

prediction = model.predict(new_df)
print("Predicted Status: ", "Pass" if prediction[0] == 1 else "Fail")

Predicted Status:  Pass
